# Agregace a spojování

Stáhneme si soubory, se kterými budeme pracovat.

In [1]:
import requests

file_names = ["u202.csv", "u203.csv", "u302.csv", "studenti.csv", "predsedajici.csv"]

for file in file_names:
    url = "https://kodim.cz/cms/assets/kurzy/python-data-1/python-pro-data-1/agregace-a-spojovani/{}".format( file )
    r = requests.get(url)
    open(file, "wb").write(r.content)
    print(f"File {file} downloaded.")

File u202.csv downloaded.
File u203.csv downloaded.
File u302.csv downloaded.
File studenti.csv downloaded.
File predsedajici.csv downloaded.


Vytvoříme pandas DataFrame pro soubory s výsledky maturitní zkoušky.

In [2]:
import pandas

u202 = pandas.read_csv("../soubory_k_praci/u202.csv")
u203 = pandas.read_csv("../soubory_k_praci/u203.csv")
u302 = pandas.read_csv("../soubory_k_praci/u302.csv")

Podíváme se na data.

In [3]:
u202

,cisloStudenta,predmet,znamka,den
0,1,Chemie,NaN,pá
1,2,Dějepis,3.0,pá
2,3,Matematika,2.0,út
3,2,Společenské vědy,2.0,pá
4,4,Biologie,1.0,pá
5,5,Dějepis,1.0,po
6,6,Fyzika,2.0,čt
7,7,Dějepis,4.0,po
8,8,Matematika,2.0,po
9,9,Dějepis,NaN,pá


## Práce s chybějícími hodnotami
anglicky _missing values_


**Co je chybějící hodnota?**
- hodnota `NaN` (Not a Number)
    - _napr. prazdna bunka v csv --> rovnou nahrazena NaN_
- `None` = zabudovaná konstanta v Pythonu 
- `pd.NA` = zatím experiment hodnota, která by měla reprezentovat chybějící hodnoty všech typů

<br>
<br>

**Co není chybějící hodnota?**
- řetězec `''` nebo `'NaN'` and `'None'`
- `inf` / `-inf` = infinity je nedefinované číslo, kladné nebo záporné

<br>
<br>

**Jak s chybějící hodnotou zacházet?**
1. Nejlepší je vždy ověření, proč údaje chybí (např. u poskytovatele dat) a pokud je to možné, zajistit jejich doplnění.
2. Nahradit chybějící hodnoty jinými hodnotami.
3. Odstranit všechny řádky s chybějícími daty z datového setu.
4. Vyčlenit je do separátního datasetu a zpracovat je zvlášť.

_Důležité je mít na paměti, že vyřazením některých řádků může dojít ke zkreslení výsledků analýzy!_

### Odstranění neúplných řádků

**Jak chybějící hodnoty najít a odstranit**



**`isnull()`** 

**`notnull()`**

Obě metody vrací hodnoty `True` nebo `False`

Vyzkoušíme si `isnull()`.

In [4]:
u202['znamka'].isnull()

## Alternativa 
# u202['znamka'].isna() # isna() se NIJAK neliší od isnull() 

0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14    False
Name: znamka, dtype: bool

Vyzkoušíme si `notnull()`.

In [5]:
u202['znamka'].notnull()

## Alternativa
# u202['znamka'].notna()

0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9     False
10     True
11     True
12     True
13     True
14     True
Name: znamka, dtype: bool

Vybereme pomocí `isnull()` jen řádky, které mají chybějící hodnoty.

In [6]:
## Najedeme je ty, kde isnull() je True
jen_null_radky = u202[u202["znamka"].isnull()]
jen_null_radky

,cisloStudenta,predmet,znamka,den
0,1,Chemie,NaN,pá
9,9,Dějepis,NaN,pá


### Další metody na práci s chybějícími hodnotami


**`dropna()`**
- vrátí datový set očištěn od chybějících dat
- `dropna(axis=0)` --> odstraní všechny řádky, které obsahují chybějící data
- `dropna(axis=1)` --> odstraní všechny sloupce, které obsahují chybějící data
- Dokumentace: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
<br>
<br>


**`fillna(x)`**
- nahradí všechna chybějící data a hodnoty hodnotou x
- Dokumentace: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html


In [7]:
## Příklady těchto metod - pojďme si to vyzkoušet
testovaci_data = u202[u202['znamka'].isnull()]
testovaci_data

,cisloStudenta,predmet,znamka,den
0,1,Chemie,NaN,pá
9,9,Dějepis,NaN,pá


In [8]:
# Odstranění řádků s dropna()
testovaci_data.dropna()

## Alternativa
# testovaci_data.dropna(axis=0)

,cisloStudenta,predmet,znamka,den


In [9]:
# Odstranění sloupců s dropna()
testovaci_data.dropna(axis=1)

,cisloStudenta,predmet,den
0,1,Chemie,pá
9,9,Dějepis,pá


In [10]:
# Nahrazení chybějících hodnot
testovaci_data.fillna('Hodnota chybí')

,cisloStudenta,predmet,znamka,den
0,1,Chemie,Hodnota chybí,pá
9,9,Dějepis,Hodnota chybí,pá


## Spojení dat

Pojďme spojit naše tři tabulky dohromady.


- Tabulky spojíme pod sebe. 
- Budeme stále mít tři sloupce.
- Počet řádků bude odpovídat počtu řádků všech tří tabulek dohromady.
- _V SQL tomu odpovídá operace `UNION`._

Použijeme naše DataFrames a očistíme je o řádky s chybějícími hodnotami

In [11]:
## Tak to pojďme vyčistit
u202 = pandas.read_csv('../soubory_k_praci/u202.csv').dropna()
u203 = pandas.read_csv('../soubory_k_praci/u203.csv').dropna()
u302 = pandas.read_csv('../soubory_k_praci/u302.csv').dropna()

**Funkce `concat`**


! Pozor: funkce rozbije index. 
<br>
<br>
Spojí totiž indexy tabulek za sebe např. 1 2 3 1 2 3 1 2 3. <br>
Pomůže nám parameter `ignore_index`

In [12]:
maturita = pandas.concat([u202, u203, u302])
maturita

,cisloStudenta,predmet,znamka,den
1,2,Dějepis,3.0,pá
2,3,Matematika,2.0,út
3,2,Společenské vědy,2.0,pá
4,4,Biologie,1.0,pá
5,5,Dějepis,1.0,po
6,6,Fyzika,2.0,čt
7,7,Dějepis,4.0,po
8,8,Matematika,2.0,po
10,10,Chemie,2.0,st
11,3,Chemie,5.0,út


In [13]:
maturita = pandas.concat([u202, u203, u302], ignore_index=True)
maturita

,cisloStudenta,predmet,znamka,den
0,2,Dějepis,3.0,pá
1,3,Matematika,2.0,út
2,2,Společenské vědy,2.0,pá
3,4,Biologie,1.0,pá
4,5,Dějepis,1.0,po
5,6,Fyzika,2.0,čt
6,7,Dějepis,4.0,po
7,8,Matematika,2.0,po
8,10,Chemie,2.0,st
9,3,Chemie,5.0,út


**Vytvoření nového sloupce**

Spojením tabulek jsme ztratili informace o čísle místnosti. 

Můžeme si ale číslo místnosti před spojením uložit do nového sloupečku.

In [14]:
u202['mistnost'] = 'u202'
u203['mistnost'] = 'u203'
u302['mistnost'] = 'u302'

maturita = pandas.concat([u202, u203, u302], ignore_index=True)

**Uložení tabulky do souboru**

In [15]:
# dataframe.to_csv('soubor.csv', index=False)

maturita.to_csv('../soubory_k_praci/maturita.csv', index=False)

Finální tabulku, kterou jsme právě vytvořili, si můžete pro kontrolu stáhnout zde: 

<https://stage.kodim.cz/cms/assets/kurzy/python-data-1/python-pro-data-1/agregace-a-spojovani/maturita.csv>

## Propojení dat

- Tabulky můžeme spojit více způsoby. 
- _V SQL tomu odpovídá operace `JOIN`._
- Výsledná tabulka bude mít více sloupců.
- Počet řádků na typu propojení.


**Typy propojení**

![Druhy propojeni](../img/type_of_join_operations.png)

**Přidáme další tabulku**

Naše výsledky byly anonymní. Pokud bychom ale chtěli vytisknout maturitní vysvědčení, potřebujeme k číslům studenta zjistit jejich jména. 

Jména najdeme v samostatné tabulce `studenti.csv`. Načtěme si jej jako `DataFrame`.

In [16]:
studenti = pandas.read_csv('../soubory_k_praci/studenti.csv')

U operace `JOIN` jsou důležité dvě věci:

- Podle jakého sloupce (nebo jakých sloupců) dvě různé tabulky propojujeme.
- Co udělat v případě, že pro nějaké řádky nemám ve druhé tabulce odpovídající hodnotu.

**Funkce `merge`**

Dokumentace: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

In [17]:
propojeny_df = pandas.merge(u202, studenti)

Ve výchozím nastavení funkce merge() ponechá pouze řádky, které mají záznamy v obou tabulkách. 

V SQL bychom tuto operaci označili jako INNER JOIN.

In [34]:
"""
Pokud by například nějaký student nebyl uvedený v tabulce se studenty, jeho maturitní výsledek by zmizel. 
U nového DataFrame bychom tedy měli zkontrolovat, zda má propojeny_df stejný počet řádků jako u202.
"""

print(u202.shape)

print(propojeny_df.shape)

propojeny_df

(13, 5)
(13, 6)


,cisloStudenta,predmet,znamka,den,mistnost,jmeno
0,2,Dějepis,3.0,pá,u202,Lukáš Jurčík
1,2,Společenské vědy,2.0,pá,u202,Lukáš Jurčík
2,3,Matematika,2.0,út,u202,Pavel Horák
3,3,Chemie,5.0,út,u202,Pavel Horák
4,4,Biologie,1.0,pá,u202,Pavel Kysilka
5,5,Dějepis,1.0,po,u202,Kateřina Novotná
6,6,Fyzika,2.0,čt,u202,Marie Krejcárková
7,7,Dějepis,4.0,po,u202,Vasil Lácha
8,8,Matematika,2.0,po,u202,Alexey Opatrný
9,10,Chemie,2.0,st,u202,Miroslav Bednář


**Tabulka s předsedy maturitních komisí**

In [35]:
preds = pandas.read_csv('../soubory_k_praci/predsedajici.csv')

preds

,datum,jmeno,den
0,20.5.2019,Marie Zuzaňáková,po
1,21.5.2019,Marie Zuzaňáková,út
2,22.5.2019,Petr Ortinský,st
3,23.5.2019,Petr Ortinský,čt
4,24.5.2019,Alena Pniáčková,pá


In [36]:
novy_propojeny_df = pandas.merge(propojeny_df, preds)
novy_propojeny_df.head()

,cisloStudenta,predmet,znamka,den,mistnost,jmeno,datum


Tentokrát jsme příliš neuspěli, výsledný DataFrame je prázdný. 

Protože v obou DataFrame máme sloupec **jmeno**, v jednom případě však jde o jméno studenta a ve druhém o jméno předsedy komise. 

Musíme říct, že chceme data spojit pouze podle sloupce den.

In [21]:
novy_propojeny_df = pandas.merge(propojeny_df, preds, on=['den'])
novy_propojeny_df

,cisloStudenta,predmet,znamka,den,mistnost,jmeno_x,datum,jmeno_y
0,2,Dějepis,3.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
1,2,Společenské vědy,2.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
2,4,Biologie,1.0,pá,u202,Pavel Kysilka,24.5.2019,Alena Pniáčková
3,3,Matematika,2.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
4,3,Chemie,5.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
5,6,Fyzika,2.0,čt,u202,Marie Krejcárková,23.5.2019,Petr Ortinský
6,10,Chemie,2.0,st,u202,Miroslav Bednář,22.5.2019,Petr Ortinský
7,10,Dějepis,5.0,st,u202,Miroslav Bednář,22.5.2019,Petr Ortinský
8,11,Matematika,1.0,st,u202,Ivana Dvořáková,22.5.2019,Petr Ortinský
9,12,Biologie,4.0,st,u202,Lenka Jarošová,22.5.2019,Petr Ortinský


**Zmizely nám řádky!**

To znamená, že funkce `merge()` **nenašla pro všechna zkoušení odpovídajícího předsedu**. 

<br>

Zkusme nyní říct funkci `merge()`, aby nám zachovala v prvním DataFrame ty řádky, pro které nenajde odpovídající záznam. 

Této operaci se v jazyce SQL říká **LEFT OUTER JOIN**. 

My ho provede tak, že funkci `merge()` jako parametr `how` zadáme hodnotu `left`.

In [22]:
novy_propojeny_df = pandas.merge(propojeny_df, preds, on=['den'], how="left") # how = "outer"
novy_propojeny_df

,cisloStudenta,predmet,znamka,den,mistnost,jmeno_x,datum,jmeno_y
0,2,Dějepis,3.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
1,2,Společenské vědy,2.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
2,3,Matematika,2.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
3,3,Chemie,5.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
4,4,Biologie,1.0,pá,u202,Pavel Kysilka,24.5.2019,Alena Pniáčková
5,5,Dějepis,1.0,po,u202,Kateřina Novotná,NaN,NaN
6,6,Fyzika,2.0,čt,u202,Marie Krejcárková,23.5.2019,Petr Ortinský
7,7,Dějepis,4.0,po,u202,Vasil Lácha,NaN,NaN
8,8,Matematika,2.0,po,u202,Alexey Opatrný,NaN,NaN
9,10,Chemie,2.0,st,u202,Miroslav Bednář,22.5.2019,Petr Ortinský


Zkusme si zobrazit ty řádky, které se nepodařilo propojit. 

Poznáme je tak, že mají prázdný sloupec datum.

Z nějakého důvodu nám nefunguje propojení v případě, že ve sloupci den je hodnota po. 

In [23]:
novy_propojeny_df[novy_propojeny_df["datum"].isnull()]

,cisloStudenta,predmet,znamka,den,mistnost,jmeno_x,datum,jmeno_y
5,5,Dějepis,1.0,po,u202,Kateřina Novotná,NaN,NaN
7,7,Dějepis,4.0,po,u202,Vasil Lácha,NaN,NaN
8,8,Matematika,2.0,po,u202,Alexey Opatrný,NaN,NaN


**funkce `strip()`**

z řetězce odstraní mezery (a další bílé znaky) na začátku a na konci

In [24]:
preds["den"] = preds["den"].str.strip()


In [41]:
#novy_propojeny_df = pandas.merge(propojeny_df, preds, on=['den'], how="left")
novy_propojeny_df = propojeny_df.merge(preds, on=['den'], how="left")
novy_propojeny_df

,cisloStudenta,predmet,znamka,den,mistnost,jmeno_x,datum,jmeno_y
0,2,Dějepis,3.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
1,2,Společenské vědy,2.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
2,3,Matematika,2.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
3,3,Chemie,5.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
4,4,Biologie,1.0,pá,u202,Pavel Kysilka,24.5.2019,Alena Pniáčková
5,5,Dějepis,1.0,po,u202,Kateřina Novotná,NaN,NaN
6,6,Fyzika,2.0,čt,u202,Marie Krejcárková,23.5.2019,Petr Ortinský
7,7,Dějepis,4.0,po,u202,Vasil Lácha,NaN,NaN
8,8,Matematika,2.0,po,u202,Alexey Opatrný,NaN,NaN
9,10,Chemie,2.0,st,u202,Miroslav Bednář,22.5.2019,Petr Ortinský


**Přejmenování sloupců**

In [42]:
novy_propojeny_df = novy_propojeny_df.rename(columns={'jmeno_x': 'jmeno', 'jmeno_y': 'predseda'})

novy_propojeny_df

,cisloStudenta,predmet,znamka,den,mistnost,jmeno,datum,predseda
0,2,Dějepis,3.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
1,2,Společenské vědy,2.0,pá,u202,Lukáš Jurčík,24.5.2019,Alena Pniáčková
2,3,Matematika,2.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
3,3,Chemie,5.0,út,u202,Pavel Horák,21.5.2019,Marie Zuzaňáková
4,4,Biologie,1.0,pá,u202,Pavel Kysilka,24.5.2019,Alena Pniáčková
5,5,Dějepis,1.0,po,u202,Kateřina Novotná,NaN,NaN
6,6,Fyzika,2.0,čt,u202,Marie Krejcárková,23.5.2019,Petr Ortinský
7,7,Dějepis,4.0,po,u202,Vasil Lácha,NaN,NaN
8,8,Matematika,2.0,po,u202,Alexey Opatrný,NaN,NaN
9,10,Chemie,2.0,st,u202,Miroslav Bednář,22.5.2019,Petr Ortinský


## Agregace

**metoda `groupby()`**

Pomocí metody groupby vyrobíme z `DataFrame` speciální objekt `DataFrameGroupBy`. 

`maturita.groupby('mistnost')`

**Agregační funkce**

In [47]:
maturita.groupby('mistnost').count()

,cisloStudenta,predmet,znamka,den
mistnost,,,,
u202,13,13,13,13
u203,13,13,13,13
u302,12,12,12,12


Další užitečné agregační funkce jsou například:

* `sum` - součet hodnot,
* `max` - maximální hodnota,
* `min` - minimální hodnota,
* `first` - první hodnota,
* `last` - poslední hodnota,
* `mean` - průměr z hodnot,
* `median` - medián z hodnot.

![Princip agregace](../img/groupby.png)

**Agregace vybraných sloupců**

In [48]:
maturita.groupby('predmet')['znamka'].mean()

predmet
Biologie            2.750000
Chemie              3.800000
Dějepis             3.200000
Fyzika              2.800000
Informatika         2.333333
Matematika          2.714286
Společenské vědy    2.250000
Zeměpis             2.000000
Name: znamka, dtype: float64

In [29]:
## Nebo více sloupců
maturita.groupby('predmet')[['znamka', 'cisloStudenta']].mean()

,znamka,cisloStudenta
predmet,,
Biologie,2.750000,13.250000
Chemie,3.800000,10.400000
Dějepis,3.200000,7.600000
Fyzika,2.800000,13.800000
Informatika,2.333333,15.500000
Matematika,2.714286,11.142857
Společenské vědy,2.250000,11.250000
Zeměpis,2.000000,15.000000


**ČTENÍ NA DOMA**

**Funkce `.agg()`**

Dokumentace: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html


In [30]:
maturita.groupby("cisloStudenta").agg({"znamka": ["max", "mean"]})

znamka     
                 max mean
cisloStudenta            
2                3.0  2.5
3                5.0  3.5
4                1.0  1.0
5                3.0  2.0
6                5.0  3.5
7                4.0  3.5
8                2.0  1.5
10               5.0  3.5
11               5.0  3.0
12               4.0  3.5
13               5.0  4.5
14               3.0  3.0
15               3.0  3.0
16               2.0  2.0
17               4.0  3.5
20               2.0  1.5
21               4.0  4.0
22               3.0  3.0
23               1.0  1.0

## Řazení

**Metoda `sort_values`**

Metoda `sort_values` standardně řadí vzestupně. 

Chceme-li řadit sestupně, zadáme jí parametr `ascending` a nastavíme ho na `False`.

In [31]:
## Řazení hodnot v Series
## https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_values.html

vysledky_predmetu = maturita.groupby('predmet')['znamka'].max() 
vysledky_predmetu.sort_values(ascending=False)

predmet
Chemie              5.0
Dějepis             5.0
Matematika          5.0
Biologie            4.0
Fyzika              4.0
Informatika         4.0
Společenské vědy    3.0
Zeměpis             3.0
Name: znamka, dtype: float64

In [32]:
## Řazení hodnot v DataFramu
## https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

maturita.sort_values(by="znamka")

,cisloStudenta,predmet,znamka,den,mistnost
31,20,Matematika,1.0,čt,u302
26,23,Informatika,1.0,po,u302
3,4,Biologie,1.0,pá,u202
4,5,Dějepis,1.0,po,u202
27,23,Společenské vědy,1.0,po,u302
29,4,Informatika,1.0,pá,u302
10,11,Matematika,1.0,st,u202
20,8,Zeměpis,1.0,po,u203
23,20,Fyzika,2.0,čt,u203
30,16,Biologie,2.0,út,u302


In [33]:
## Řazení hodnot v DataFramu s Multiindexem

vysledky_studenty = maturita.groupby("cisloStudenta").agg({"znamka": ["mean"]})
vysledky_studenty.sort_values([("znamka", "mean")])

,znamka
,mean
cisloStudenta,
23,1.0
4,1.0
20,1.5
8,1.5
5,2.0
16,2.0
2,2.5
11,3.0
